### aim: 

### date: 

In [2]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [3]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [4]:
import numpy as np
from cmocean import cm
import cartopy as cp
import cartopy.crs as ccrs
import netCDF4 as nc
import matplotlib.pyplot as plt
import xarray as xr

%matplotlib inline
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')
import cartopy.feature as cfeature
from importlib import reload
import matplotlib.path as mpath
import glob
import pickle
import pandas as pd
import seawater
import time
plt.rcParams.update({'font.size': 13})
font = {'family' : 'normal',
'weight' : 'normal',
'size'   : 13}
plt.rcParams['text.usetex'] = True
plt.rc('font', **font)

# UKESM 

In [29]:
tdir = '/gpfs/data/greenocean2/software/products/windsFromComponents/symlinked_hrly_UKESM/newdaily/'
cdir = '/gpfs/data/greenocean2/software/products/windsFromComponents/dailyStandard/UKESM/'
mons = ['01','02','03','04','05','06','07','08','09','10','11','12']
scens = ['1B']

ex = False
if ex:
    for s in scens:
        for y in range(1940,2024):
            print(y)
            for i in range(0,12):
                uwi = xr.open_dataset(f'{tdir}scen{s}_uwnd_y{y}m{mons[i]}_rg.nc')
                vwi = xr.open_dataset(f'{tdir}scen{s}_vwnd_y{y}m{mons[i]}_rg.nc')

                wspd = xr.ufuncs.sqrt(uwi.uwind10m**2 + vwi.vwind10m**2)
                wspd.name = 'wspd10m'
                wb = xr.merge([uwi, vwi, wspd])
                wb = wb.rename({"uwind10m": "u10m"})
                wb = wb.rename({"vwind10m": "v10m"})
                savenam = f'UKESM_wind_daily_1x1_{y}m{mons[i]}.nc'
                print(savenam)
                wb.attrs = {'howmade': 'https://nettle-pajama-b85.notion.site/Winds-eval-paper-notes-76fc22d90ce14e23b8247ba370b15265?pvs=4'}
                wb.to_netcdf(f'{cdir}{savenam}')


In [35]:
w = xr.open_dataset(f'{cdir}UKESM_wind_daily_1x1_{1948}m{mons[0]}.nc')


<xarray.Dataset>
Dimensions:       (lat: 180, lon: 360, time_counter: 30)
Coordinates:
  * time_counter  (time_counter) object 1948-01-01 00:00:00 ... 1948-01-30 00...
  * lon           (lon) float64 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
  * lat           (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
Data variables:
    u10m          (time_counter, lat, lon) float32 ...
    v10m          (time_counter, lat, lon) float32 ...
    wspd10m       (time_counter, lat, lon) float32 ...
Attributes:
    howmade:  https://nettle-pajama-b85.notion.site/Winds-eval-paper-notes-76...

## NCEP-NCAR

In [60]:
yst = 1948; yen = 2024
tdir = '/gpfs/data/greenocean2/software/products/windsFromComponents/NCEP-NCAR/daily/'
cdir = '/gpfs/data/greenocean2/software/products/windsFromComponents/dailyStandard/'

ex = False

if ex:

    for i in range(yst,yen):

        print(i)
        
        times = pd.date_range(f"{i}/01/01",f"{i+1}/01/01",freq='D',closed='left')

        ws = xr.open_dataset(f'{tdir}wspd.10m.gauss.{i}_daily_rg.nc')
        wu = xr.open_dataset(f'{tdir}uwnd.10m.gauss.{i}_daily_rg.nc')
        wv = xr.open_dataset(f'{tdir}vwnd.10m.gauss.{i}_daily_rg.nc')
        
        wb = xr.merge([ws, wu, wv])
        
        wb = wb.rename({"time": "time_counter"})
        wb['time_counter'] = times
        wb = wb.rename({"uwnd": "u10m"})
        wb = wb.rename({"vwnd": "v10m"})
        wb = wb.rename({"windspeed": "wspd10m"})

        savenam = f'NCEP-NCAR_wind_daily_1x1_{i}.nc'
        print(savenam)
        wb.attrs = {'howmade': 'https://nettle-pajama-b85.notion.site/Winds-eval-paper-notes-76fc22d90ce14e23b8247ba370b15265?pvs=4'}
        wb.to_netcdf(f'{cdir}/NCEP-NCAR/{savenam}')

## NCEP-DOE

In [80]:
ex = False
if ex:
    
    tdir = '/gpfs/data/greenocean2/software/products/windsFromComponents/NCEP-DOE/'
    uw = xr.open_dataset(f'{tdir}/data_uwnd_rg.nc')
    vw = xr.open_dataset(f'{tdir}/data_vwnd_rg.nc')
    ww = xr.open_dataset(f'{tdir}/data_windspeed_rg.nc')

    for yr in range(1979,2024):
        print(yr)
        uwd = uw.sel(time=slice(f'{yr}-01-01', f'{yr}-12-31')).isel(level = 0)
        vwd = vw.sel(time=slice(f'{yr}-01-01', f'{yr}-12-31')).isel(level = 0)
        wwd = ww.sel(time=slice(f'{yr}-01-01', f'{yr}-12-31')).isel(level = 0)
        wb = xr.merge([uwd, vwd, wwd])
        wb = wb.drop('level')
        wb = wb.rename({"uwnd": "u10m"})
        wb = wb.rename({"vwnd": "v10m"})
        wb = wb.rename({"windspeed": "wspd10m"})
        wb = wb.rename({"time": "time_counter"})

        savenam = f'NCEP-DOE_wind_daily_1x1_{yr}.nc'
        print(savenam)
        wb.attrs = {'howmade': 'https://nettle-pajama-b85.notion.site/Winds-eval-paper-notes-76fc22d90ce14e23b8247ba370b15265?pvs=4'}
        wb.to_netcdf(f'{cdir}/NCEP-DOE/{savenam}')

<xarray.Dataset>
Dimensions:    (lat: 180, lon: 360, time: 365)
Coordinates:
  * time       (time) datetime64[ns] 1979-01-01T12:00:00 ... 1979-12-31T12:00:00
  * lon        (lon) float64 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
Data variables:
    uwnd       (time, lat, lon) float32 ...
    vwnd       (time, lat, lon) float32 ...
    windspeed  (time, lat, lon) float32 ...

In [81]:
w = xr.open_dataset('/gpfs/data/greenocean2/software/products/windsFromComponents/ERA5_v2024/scripts/daily/10m_windspeed_ERA5_1958_daily.nc')
w

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 365)
Coordinates:
  * time       (time) datetime64[ns] 1958-01-01 1958-01-02 ... 1958-12-31
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
Data variables:
    windspeed  (time, latitude, longitude) float32 ...